# Music Recommendation System
This notebook demonstrates a hybrid recommendation system using collaborative and content-based filtering.

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")


##  Load Data

In [ ]:
songs_df = pd.read_csv('songs_metadata.csv')
interactions_df = pd.read_csv('user_interactions.csv')
songs_df.head()


##  Collaborative Filtering (Using Surprise SVD)

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(interactions_df[['user_id', 'song_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2)

model = SVD()
model.fit(trainset)

predictions = model.test(testset)

from surprise.accuracy import rmse
rmse(predictions)


##  Content-Based Filtering

In [ ]:
features = songs_df[['tempo', 'energy']]
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

similarity_matrix = cosine_similarity(features_scaled)
similarity_df = pd.DataFrame(similarity_matrix, index=songs_df['title'], columns=songs_df['title'])

# Show similar songs to "Dreaming High"
similarity_df['Dreaming High'].sort_values(ascending=False)


##  Hybrid Recommendation

In [ ]:
def recommend_songs(user_id, top_n=3):
    user_history = interactions_df[interactions_df['user_id'] == user_id]['song_id'].tolist()
    all_songs = songs_df['song_id'].tolist()
    recommendations = []

    for song_id in all_songs:
        if song_id not in user_history:
            pred = model.predict(user_id, song_id)
            recommendations.append((song_id, pred.est))
    
    recommendations.sort(key=lambda x: x[1], reverse=True)
    top_recs = [song for song, _ in recommendations[:top_n]]
    return songs_df[songs_df['song_id'].isin(top_recs)][['title', 'artist', 'genre']]

recommend_songs('u3')


##  Conclusion
We built a simple hybrid recommender system using collaborative filtering (SVD) and content-based filtering (cosine similarity of song features).